In [1]:
import random
import numpy as np


#模拟生成数据的函数
def get_O(T):
    #这里是真实数据,也就是训练的目标
    A = np.array([[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]])
    B = np.array([[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]])
    pi = np.array([0.2, 0.4, 0.4])

    #状态链
    I = []

    #输出链
    O = []

    #循环的次数就是输出链的长度
    for i in range(T):

        #状态转移的随机数
        r = random.random()

        #如果是第一次的话,用pi的概率确定第一个a
        if i == 0:
            a = pi

        #否则,用上次a的概率求下一个a的概率
        if r < a[0]:
            idx = 0
        elif r < a[0] + a[1]:
            idx = 1
        else:
            idx = 2

        a = A[idx]
        b = B[idx]
        I.append(idx)

        #计算输出的随机数
        r = random.random()

        if r < b[0]:
            O.append(0)
        else:
            O.append(1)

    return I, O


get_O(5)

([1, 0, 2, 1, 1], [1, 0, 1, 0, 0])

In [2]:
T = 1000

#状态链
I_s = np.empty((T, T))

#输出链
O_s = np.empty((T, T))

for i in range(T):
    I_s[i], O_s[i] = get_O(T)

I_s, O_s

(array([[2., 2., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [2., 1., 2., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 2., 2., 2.],
        [2., 2., 0., ..., 1., 1., 1.],
        [2., 2., 1., ..., 1., 0., 0.]]), array([[0., 1., 0., ..., 0., 1., 1.],
        [0., 1., 1., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 1., 0.],
        ...,
        [1., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.]]))

In [3]:
#A是状态转换概率矩阵,标识不同状态之间相互转换的概率
A = np.zeros((3, 3))

#B是输出概率矩阵,标识在不同状态下各个输出的概率
B = np.zeros((3, 2))

#pi是第一次会进去某个状态的概率
pi = np.zeros(3)

A, B, pi

(array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]), array([[0., 0.],
        [0., 0.],
        [0., 0.]]), array([0., 0., 0.]))

In [4]:
#pi的概率直接拿I_s的第一列计算就行了
pi[0] = np.sum([I_s[:, 0] == 0]) / T
pi[1] = np.sum([I_s[:, 0] == 1]) / T
pi[2] = np.sum([I_s[:, 0] == 2]) / T
pi

array([0.181, 0.412, 0.407])

In [5]:
a_from_0_to_any = 0
a_from_0_to_0 = 0
a_from_0_to_1 = 0
a_from_0_to_2 = 0

a_from_1_to_any = 0
a_from_1_to_0 = 0
a_from_1_to_1 = 0
a_from_1_to_2 = 0

a_from_2_to_any = 0
a_from_2_to_0 = 0
a_from_2_to_1 = 0
a_from_2_to_2 = 0

#遍历所有的状态
for i in range(T):
    #最后一个状态不用看了,因为他不会转变成很别的状态
    for j in range(T - 1):
        a = I_s[i, j]

        #处理当前状态是0的情况
        if a == 0:
            a_from_0_to_any += 1

            #看下一个状态是谁,就给谁的转换次数+1就行了
            a_next = I_s[i, j + 1]
            if a_next == 0:
                a_from_0_to_0 += 1
            if a_next == 1:
                a_from_0_to_1 += 1
            if a_next == 2:
                a_from_0_to_2 += 1

        if a == 1:
            a_from_1_to_any += 1
            a_next = I_s[i, j + 1]
            if a_next == 0:
                a_from_1_to_0 += 1
            if a_next == 1:
                a_from_1_to_1 += 1
            if a_next == 2:
                a_from_1_to_2 += 1

        if a == 2:
            a_from_2_to_any += 1
            a_next = I_s[i, j + 1]
            if a_next == 0:
                a_from_2_to_0 += 1
            if a_next == 1:
                a_from_2_to_1 += 1
            if a_next == 2:
                a_from_2_to_2 += 1

a_from_0_to_any, a_from_0_to_0, a_from_0_to_1, a_from_0_to_2

(331974, 165697, 65943, 100334)

In [6]:
#这里的工作就很简单了,以A[0,0]为例,A0转换到A0的概率,就是A0到A0的次数 / A0到任何状态的次数
A[0][0] = a_from_0_to_0 / a_from_0_to_any
A[0][1] = a_from_0_to_1 / a_from_0_to_any
A[0][2] = a_from_0_to_2 / a_from_0_to_any

A[1][0] = a_from_1_to_0 / a_from_1_to_any
A[1][1] = a_from_1_to_1 / a_from_1_to_any
A[1][2] = a_from_1_to_2 / a_from_1_to_any

A[2][0] = a_from_2_to_0 / a_from_2_to_any
A[2][1] = a_from_2_to_1 / a_from_2_to_any
A[2][2] = a_from_2_to_2 / a_from_2_to_any

A

array([[0.49912644, 0.19863905, 0.30223451],
       [0.29932175, 0.5013634 , 0.19931485],
       [0.19979381, 0.30027362, 0.49993257]])

In [7]:
b_0_out_any = 0
b_0_out_0 = 0
b_0_out_1 = 0

b_1_out_any = 0
b_1_out_0 = 0
b_1_out_1 = 0

b_2_out_any = 0
b_2_out_0 = 0
b_2_out_1 = 0

#遍历所有的状态
for i in range(T):
    for j in range(T):
        a = I_s[i, j]
        b = O_s[i, j]

        #处理当前状态是0的情况
        if a == 0:
            b_0_out_any += 1

            #看b输出是谁,就给谁+1就行了
            if b == 0:
                b_0_out_0 += 1
            if b == 1:
                b_0_out_1 += 1

        if a == 1:
            b_1_out_any += 1

            #看b输出是谁,就给谁+1就行了
            if b == 0:
                b_1_out_0 += 1
            if b == 1:
                b_1_out_1 += 1

        if a == 2:
            b_2_out_any += 1

            #看b输出是谁,就给谁+1就行了
            if b == 0:
                b_2_out_0 += 1
            if b == 1:
                b_2_out_1 += 1

b_0_out_any, b_0_out_0, b_0_out_1

(332324, 166071, 166253)

In [8]:
#这里的工作就很简单了,以B[0,0]为例,B0输出0的概率,就是B0输出0的次数 / B0输出任何结果的次数
B[0][0] = b_0_out_0 / b_0_out_any
B[0][1] = b_0_out_1 / b_0_out_any

B[1][0] = b_1_out_0 / b_1_out_any
B[1][1] = b_1_out_1 / b_1_out_any

B[2][0] = b_2_out_0 / b_2_out_any
B[2][1] = b_2_out_1 / b_2_out_any

B

array([[0.49972617, 0.50027383],
       [0.39972129, 0.60027871],
       [0.70001138, 0.29998862]])